In [3]:
from datasets import load_dataset
import pandas as pd
import chess
import numpy as np
import tensorflow as tf

subset = load_dataset("laion/strategic_game_chess", data_files="chess_game_0001.parquet")
df = subset["train"].to_pandas()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
df_moves = df["Moves"].apply(lambda x: [str(i) for i in x])

In [5]:
moves_array = np.array(df_moves)

In [6]:
boards = []
moves = []
for board_moves in moves_array[:10000]:
    board = chess.Board()
    board_states = []
    board_states_labels = []
    for move_str in board_moves[:10]:
        move = chess.Move.from_uci(move_str)
        move_from = move.from_square
        move_to = move.to_square
        board_states_labels.append((move_from, move_to))
        board_states.append(board.copy())
        board.push(move)
    white_states = board_states[::2]
    white_states_labels = board_states_labels[::2]
    moves.append(white_states_labels)
    boards.append(white_states)

In [ ]:
moves

In [7]:
data = np.array(boards).astype(str)

In [31]:
on_hot_mapping = {".": np.array([0,0,0,0,0,0]),
                  "P": np.array([1,0,0,0,0,0]),
                  "N": np.array([0,1,0,0,0,0]),
                  "B": np.array([0,0,1,0,0,0]),
                  "R": np.array([0,0,0,1,0,0]),
                  "Q": np.array([0,0,0,0,1,0]),
                  "K": np.array([0,0,0,0,0,1]),
                  "p": np.array([-1,0,0,0,0,0]),
                  "n": np.array([0,-1,0,0,0,0]),
                  "b": np.array([0,0,-1,0,0,0]),
                  "r": np.array([0,0,0,-1,0,0]),
                  "q": np.array([0,0,0,0,-1,0]),
                  "k": np.array([0,0,0,0,0,-1])}
games_data = []
games_labels = []
for i in range(len(data)):
    for j in range(len(data[i])):
        data_split = np.array([row.split() for row in data[i][j].split("\n")])
        labels_split = tf.keras.utils.to_categorical(moves[i][j], num_classes=64)
        data_encoded_state = np.zeros([8, 8, 6])
        for char, encoding in on_hot_mapping.items():
                data_encoded_state[data_split == char] = encoding
        games_data.append(np.array(data_encoded_state))
        games_labels.append(labels_split)

train_data = tf.constant(games_data[0 : int(len(games_data) * .70)])
train_labels = tf.constant(games_labels[0 : int(len(games_labels) * .70)])
test_data = tf.constant(games_data[int(len(games_data) * .70) : len(games_data)])
test_labels = tf.constant(games_labels[int(len(games_labels) * .70) : len(games_labels)])

In [32]:
train_data.shape, train_labels.shape, test_data.shape, test_labels.shape

(TensorShape([35000, 8, 8, 6]),
 TensorShape([35000, 2, 64]),
 TensorShape([15000, 8, 8, 6]),
 TensorShape([15000, 2, 64]))

In [28]:
input_layer = tf.keras.layers.Input(shape=(8,8,6))
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', strides=1)(input_layer)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', strides=1)(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same', strides=1)(x)
x = tf.keras.layers.Flatten()(x)
output1 = tf.keras.layers.Dense(64, activation="softmax", name="output1")(x)
output2 = tf.keras.layers.Dense(64, activation="softmax", name="output2")(x)

baseline_model = tf.keras.Model(inputs=input_layer, outputs=[output1, output2])

baseline_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy', 'accuracy'])

In [17]:
baseline_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 8, 8, 6)        │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 8, 8, 32)       │          1,760 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 8, 8, 64)       │         18,496 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 8, 8, 64)       │         36,928 │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ flatten (Flatten)         │ (None, 4096)           │              0 │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ output1 (Dense)           │ (None, 64)             │        262,208 │ flatten[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ output2 (Dense)           │ (None, 64)             │        262,208 │ flatten[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 581,600 (2.22 MB)

 Trainable params: 581,600 (2.22 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
bsm_history = baseline_model.fit(train_data, [train_labels[:,0,:], train_labels[:,1,:]], epochs=10, validation_split=0.1)

In [35]:
bsm_predict = baseline_model.predict(test_data)

469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


In [46]:
move_from_pred = np.argmax(bsm_predict[0], axis=-1)
move_to_pred = np.argmax(bsm_predict[1], axis=-1)

In [62]:
move_from_pred[0], move_to_pred[0]

(12, 28)